In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

# Adjust display options
# pd.set_option('display.max_rows', None)  # Show all rows
# pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_colwidth', None)  # Show full column width

df = pd.read_excel('../Data/legendary_lord_defeat_traits.xlsx')

In [24]:
# Define the labeling criteria
def label_trait(effect):
    # Define criteria for 'strong'
    strong_keywords = ['+30%', '+40%', '+50%', 'immense', 'huge', 'enormous', 'greatly', '+20%', '+25%', '+15%']
    # Define criteria for 'weak'
    weak_keywords = ['+2%', '+5%', '+1%', 'slight', 'minor', 'small', '+10%', '+8%', '+6%']
    # Define criteria for 'balanced'
    balanced_keywords = ['+10%', '+12%', 'moderate', '+15%', '+8%', 'both', 'combination']

    # Convert to lower case for case-insensitive matching
    effect_lower = effect.lower()
    
    # Apply the criteria to classify the effect
    if any(keyword in effect_lower for keyword in strong_keywords):
        return 'strong'
    elif any(keyword in effect_lower for keyword in weak_keywords):
        return 'weak'
    elif any(keyword in effect_lower for keyword in balanced_keywords):
        return 'balanced'
    else:
        # Default to balanced if no strong/weak indicators are found
        return 'balanced'

# Apply the labeling function to the DataFrame
df['Label'] = df['Effect'].apply(label_trait)

In [25]:
df

,Trait,Lord that must be defeated,Effect,Description,Label
0,Beastscourge,Khazrak the One-Eye,Ambush defence chance: +20% Chance of intercep...,His single eye will not avail the Lord of Beas...,strong
1,Crowreaver,Malagor the Dark Omen,Leadership: +10 when fighting against Beastmen...,The despoiler becomes the despoiled.,balanced
2,Skullslasher,Morghur the Shadowgave,Missile resistance: 15%,"Faced with an opponent of the right calibre, e...",balanced
3,"Brass Bull, Glass Jaw",Taurox the Brass Bull,Armour-piercing weapon damage: +20 Charge bon...,"As it turns out, you don't always need an immo...",strong
4,Kingslayer,Louen Leoncoeur,Charge bonus: +10 Recruitment cost: -20% for c...,"This time, the king might well be dead. Oh wel...",balanced
...,...,...,...,...,...
95,Conjoined Seperated,Vilitch the Curseling,Winds of Magic power reserve change: +10% when...,Neither Thomin's strength nor Vilitch's sorcer...,weak
96,Destroyer of Dryads,Drycha,Immune to Athel Loren attrition ( Lord's army)...,The forest's most embittered wargrove could no...,balanced
97,Tree Surgeon,Durthu,Enables flaming attacks when fighting against ...,Even the mightiest tree in the forest can be f...,balanced
98,Wildhunter,Orion,Leadership: +4 during forest battles Melee at...,"Everyone has a weakness, however well hidden; ...",balanced
